## Download and virtualize dataset

In [ ]:
from pathlib import Path

import earthaccess
from common import earthaccess_args

### Setup earthaccess query parameters

In [ ]:
dataset = "gpm_imerg"
dataset_args = earthaccess_args[dataset]
concept_id = dataset_args["concept_id"]
filename = dataset_args["filename"]
variable = dataset_args["variable"]

### Authenticate via earthaccess

In [ ]:
earthaccess.login()

### Download dataset

In [ ]:
results = earthaccess.search_data(
    concept_id=concept_id, count=1, temporal=("2002-06-01", "2002-06-01")
)
fp = earthaccess.download(results, "earthaccess_data")[0]

### Virtualize dataset

In [ ]:
from icechunk import (  # noqa
    IcechunkStore,
    S3Credentials,
    StorageConfig,
    StoreConfig,
    VirtualRefConfig,
)
from virtualizarr import open_virtual_dataset  # noqa
from virtualizarr.writers.icechunk import dataset_to_icechunk  # noqa

In [ ]:
def virtualize_dataset(local_fp):
    """Create a virtual reference file for a dataset"""

    def local_to_s3_url(old_local_path: str) -> str:
        """Replace local path to s3 uri for all chucks"""

        new_s3_bucket_url = Path("/".join(s3_uri.split("/")[1:-1]))
        filename = Path(old_local_path).name
        new_path = f"s3:/{str(new_s3_bucket_url / filename)}"
        return new_path

    s3_uri = results[0].data_links(access="direct")[0]
    virtual_ds = open_virtual_dataset(str(local_fp), indexes={})
    virtual_ds = virtual_ds.virtualize.rename_paths(local_to_s3_url)
    virtual_ds = virtual_ds[[variable]]
    return virtual_ds.drop_vars("time")

In [ ]:
virtual_ds = virtualize_dataset(fp)

### Store virtual dataset as kerchunk reference


In [ ]:
s3_uri = results[0].data_links(access="direct")[0]
if dataset == "gpm_merg":
    output_fp = f"earthaccess_data/{s3_uri.split('/')[-1][:-4]}.json"
else:
    output_fp = f"earthaccess_data/{s3_uri.split('/')[-1][:-3]}.json"
virtual_ds.virtualize.to_kerchunk(output_fp, format="json")

### Store dataset using icechunk


In [ ]:
s3_creds = earthaccess.get_s3_credentials(daac=dataset_args["daac"])
credentials = S3Credentials(
    access_key_id=s3_creds["accessKeyId"],
    secret_access_key=s3_creds["secretAccessKey"],
    session_token=s3_creds["sessionToken"],
)
storage = StorageConfig.s3_from_env(
    bucket="nasa-veda-scratch",
    prefix="icechunk/test-weight-caching/reference",
    region="us-west-2",
)
config = StoreConfig(
    virtual_ref_config=VirtualRefConfig.s3_from_config(credentials=credentials),
)
store = IcechunkStore.open_or_create(storage=storage, config=config, mode="w")
dataset_to_icechunk(virtual_ds, store)
store.commit("Create  IMERG refenence")